# MusicXML Visualization

This notebook is phase two of this project, attempting to take what I've learned
about gathering data and visualize it.

## Prep

### Imports

In [131]:
# imports and constants
import music21
from pprint import pprint
from difficulty import *
from type21 import *
import warnings
import os
import numpy as np
import pandas as pd
# import ScoreProcessingFunctions as spf

### Parse MusicXML File

In [132]:
score = music21.converter.parse("../music/xml/musicalion/beethoven_symphony_5_mvt_2.musicxml")

## Extraction and Organization

In [133]:
currTempo = None
score_list = []

for part in score:
    try:
        instrument = part.getInstrument().instrumentName
    except:
        print(part)
        continue
    for element in part.flatten():
        record_in_data_frame = True
        # define items to be added to the data frame
        measure = element.measureNumber
        note_start = None
        note_end = None
        note_obj = None
        pitch = None
        volume = None
        tempo = None
        note_type = None

        if isChord(element):
            sortedNotes = element.sortAscending()
            note = sortedNotes[-1]
            note_start = note.offset
            note_end = note.offset + note.quarterLength
            note_obj = note
            pitch = note.pitch.ps
            volume = note.volume.realized
            tempo = currTempo
            note_type = type(note)
            # for note in element:
            #     note_start = note.offset
            #     note_end = note.offset + note.quarterLength
            #     note_obj = note
            #     pitch = note.pitch.ps
            #     volume = note.volume.realized
            #     tempo = currTempo
            #     note_type = type(note)
        elif isNote(element):
            note_start = element.offset
            note_end = element.offset + element.quarterLength
            note_obj = element
            pitch = element.pitch.ps
            volume = element.volume.realized
            note_type = type(element)
        elif type(element) == music21.tempo.MetronomeMark:
            record_in_data_frame = False
            currTempo = element.number
        elif isRest(element):
            note_start = element.offset
            note_end = element.offset + element.quarterLength
            note_type = type(element)
        if record_in_data_frame:
            score_list.append([measure, note_start, note_end, note_obj, pitch, volume, currTempo, note_type, instrument])


<music21.text.TextBox '-  #  -'>
<music21.layout.ScoreLayout>
<music21.layout.StaffGroup <music21.stream.PartStaff P8-Staff1><music21.stream.PartStaff P8-Staff2>>
<music21.layout.StaffGroup <music21.stream.PartStaff P10-Staff1><music21.stream.PartStaff P10-Staff2>>
<music21.layout.StaffGroup <music21.stream.Part Flauti><music21.stream.Part Oboi><music21.stream.Part Clarinetti in B><music21.stream.Part Fagotti><music21.stream.Part Corni in C><music21.stream.Part Trombe in C><music21.stream.Part Timpani in C. G.><music21.stream.PartStaff P8-Staff1><music21.stream.PartStaff P8-Staff2><music21.stream.Part Viola><music21.stream.PartStaff P10-Staff1><music21.stream.PartStaff P10-Staff2>>


In [134]:
df = pd.DataFrame(score_list, columns=['measure', 'note_start', 'note_end','note', 'pitch', 'volume', 'tempo', 'note_type', 'instrument'])
df

,measure,note_start,note_end,note,pitch,volume,tempo,note_type,instrument
0,0.0,None,None,None,NaN,NaN,None,None,Midi_74
1,NaN,None,None,None,NaN,NaN,None,None,Midi_74
2,0.0,None,None,None,NaN,NaN,None,None,Midi_74
3,0.0,None,None,None,NaN,NaN,None,None,Midi_74
4,0.0,None,None,None,NaN,NaN,None,None,Midi_74
...,...,...,...,...,...,...,...,...,...
12378,246.0,368.5,369.0,None,NaN,NaN,None,<class 'music21.note.Rest'>,Midi_44
12379,246.0,369.0,369.5,None,NaN,NaN,None,<class 'music21.note.Rest'>,Midi_44
12380,247.0,369.5,370.0,<music21.note.Note A->,44.0,1.0,None,<class 'music21.note.Note'>,Midi_44
12381,247.0,370.0,370.5,None,NaN,NaN,None,<class 'music21.note.Rest'>,Midi_44


In [135]:
# get rows where instrument is bassoon and note_type is note or rest
# df1 = df.loc[(df['instrument'] == 'Midi_71') & (df['note_type'].isin([music21.note.Note, music21.note.Rest]))]
df1 = df.loc[(df['instrument'] == 'Midi_71') & (df['note_type'] == music21.note.Note)]
# mark the duplicates of note_start column first occurence
df1['is_duplicate'] = df1.duplicated(subset='note_start', keep='first')
df_first_bassoon = df1.loc[df1['is_duplicate'] == False]
df_first_bassoon.head(10)
df_first_bassoon.tail(10)
# df1.plot(x="measure", figsize=(20, 20), subplots=True)

/var/folders/67/z2c_5yw12tbcjdhcbfkqwg0c0000gn/T/ipykernel_48443/2639807010.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['is_duplicate'] = df1.duplicated(subset='note_start', keep='first')


,measure,note_start,note_end,note,pitch,volume,tempo,note_type,instrument,is_duplicate
4504,241.0,1085/3,2171/6,<music21.note.Note C>,60.0,0.992124,None,<class 'music21.note.Note'>,Midi_71,False
4505,241.0,2171/6,362,<music21.note.Note E->,63.0,0.992124,None,<class 'music21.note.Note'>,Midi_71,False
4510,242.0,363.0,363.375,<music21.note.Note A->,56.0,1.000000,None,<class 'music21.note.Note'>,Midi_71,False
4511,242.0,363.375,363.5,<music21.note.Note C>,60.0,1.000000,None,<class 'music21.note.Note'>,Midi_71,False
4514,243.0,363.5,365.0,<music21.note.Note E->,63.0,1.000000,None,<class 'music21.note.Note'>,Midi_71,False
4516,244.0,365.0,365.5,<music21.note.Note C>,60.0,1.000000,None,<class 'music21.note.Note'>,Midi_71,False
4524,245.0,367.5,367.875,<music21.note.Note A->,56.0,0.992124,None,<class 'music21.note.Note'>,Midi_71,False
4525,245.0,367.875,368.0,<music21.note.Note C>,60.0,0.992124,None,<class 'music21.note.Note'>,Midi_71,False
4528,246.0,368.0,369.5,<music21.note.Note E->,63.0,1.000000,None,<class 'music21.note.Note'>,Midi_71,False
4530,247.0,369.5,370.0,<music21.note.Note A->,56.0,1.000000,None,<class 'music21.note.Note'>,Midi_71,False


In [136]:
# define a function that takes in a row and returns the value for the new column
def rolling_sum(row):
    # create a rolling window that includes the current row and 4 rows above and below it
    window = df_first_bassoon['note_start'].rolling(9, center=True, min_periods=1)
    # exclude NaN values from the window and calculate the rolling sum
    # return window.apply(calculate_difficulty).iloc[row.name]
    # print(row)
    # print what the window looks like using apply()
    # window.apply(lambda x: print(x[~np.isnan(x)]))

    return window.apply(lambda x: x[~np.isnan(x)].sum()).iloc[row.name]
    return window.apply(calculate_difficulty).iloc[row.name]

# translate the calculate_difficulty function into a function for use with pandas
def calculate_difficulty(x):
    return x[~np.isnan(x)].sum()
    print(x)
    return calculate_difficulty(x['note_start'], x['note_end'], x['pitch'], x['volume'], x['tempo'])

In [137]:
# reset the index of the dataframe
df_first_bassoon = df_first_bassoon.reset_index(drop=True)
# use apply() to create a new column based on the rolling sum of the values from other rows
df_first_bassoon['rolling_sum'] = df_first_bassoon.apply(rolling_sum, axis=1)
# restore the original index
df_first_bassoon.head(50)

,measure,note_start,note_end,note,pitch,volume,tempo,note_type,instrument,is_duplicate,rolling_sum
0,8.0,12.0,12.375,<music21.note.Note A->,56.0,0.496062,None,<class 'music21.note.Note'>,Midi_71,False,65.875
1,8.0,12.375,12.5,<music21.note.Note C>,60.0,0.496062,None,<class 'music21.note.Note'>,Midi_71,False,81.250
2,9.0,12.5,14.0,<music21.note.Note E->,63.0,0.992124,None,<class 'music21.note.Note'>,Midi_71,False,96.750
3,10.0,14.0,14.5,<music21.note.Note C>,60.0,0.496062,None,<class 'music21.note.Note'>,Midi_71,False,113.750
4,10.0,15.0,15.375,<music21.note.Note A->,68.0,0.496062,None,<class 'music21.note.Note'>,Midi_71,False,132.250
5,10.0,15.375,15.5,<music21.note.Note A->,68.0,0.496062,None,<class 'music21.note.Note'>,Midi_71,False,140.250
6,11.0,15.5,17.0,<music21.note.Note G>,67.0,0.992124,None,<class 'music21.note.Note'>,Midi_71,False,148.375
7,12.0,17.0,18.5,<music21.note.Note F>,65.0,0.496062,None,<class 'music21.note.Note'>,Midi_71,False,156.875
8,13.0,18.5,20.0,<music21.note.Note C>,60.0,0.496062,None,<class 'music21.note.Note'>,Midi_71,False,164.375
9,14.0,20.0,20.5,<music21.note.Note E->,63.0,0.496062,None,<class 'music21.note.Note'>,Midi_71,False,172.375
